In [9]:
import yfinance as yf
import pandas as pd
from datetime import datetime
from datetime import timedelta
import statsmodels.api as sm

# Estimate Event Date Period

In [ ]:
file = r"event_data.xlsx"
df = pd.read_excel(file)

In [24]:
event_start_list = []
even_end_list = []

for value in df["Eventdate"]:
    try:
        date = datetime.strptime(value, "%Y-%m-%d")
        event_window_start = (date - timedelta(days=206)).date()
        event_window_close = (date - timedelta(days=6)).date()
        event_start_list.append(event_window_start)
        even_end_list.append(event_window_close)
    except Exception as e:
        print(e)

df["Estimate_event_date_start"] = event_start_list
df["Estimate_event_date_end"] = even_end_list  

df.drop(columns=["Event date1"], inplace=True)
df.to_excel("event_study_data.xlsx", index=False)

## Market Model

In [ ]:
new_file = r"event_study_data.xlsx"
df_new = pd.read_excel(new_file)

In [ ]:
event_alpha_list = []
event_beta_list = []

for ticker, event_start, event_end in zip(df_new["Ticker"], df_new["Estimate_event_date_start"], df_new["Estimate_event_date_end"]):
    event_start = event_start.date() - timedelta(days=1) # In order to calculate the Return, period t-1's data is a must.  
    event_end = event_end.date() + timedelta(days=1) # yf's history "end" parameter is exclusive.
    
    # Get Company's data 
    company = yf.Ticker(ticker)
    df_company = company.history(start=event_start, end=event_end, auto_adjust=True)  
    company_price = df_company["Close"].pct_change()
    company_price.dropna(inplace=True) # First value can't be calcuated, will return NaN

    # Get S&P500 Index data 
    sp = yf.Ticker("^GSPC")
    df_sp = sp.history(start=event_start, end=event_end, auto_adjust=True)  
    sp_price = df_sp["Close"].pct_change() # calculate Return for each day
    sp_price.dropna(inplace=True)

    # Integrate data into Dataframe
    df = pd.DataFrame({"x": sp_price, "y": company_price})

    # Market Model OLS
    # X need to be 2D array as a matrix, and add constant manully
    X = sm.add_constant(df[["x"]])
    # y can be Series
    y = df["y"]

    model = sm.OLS(y, X).fit()

    # Get the parameter needed
    paraeters = model.params 
    alpha_parameter = paraeters.iloc[0].round(10)
    beta_parameter = paraeters.iloc[1].round(10)

    event_alpha_list.append(alpha_parameter)
    event_beta_list.append(beta_parameter)

In [ ]:
df_new["alpha"] = event_alpha_list
df_new["beta"] = event_beta_list

In [63]:
df_new.to_excel("event_data_for_abnormal_return.xlsx", index=False)